In [1]:
import pandas as pd
import gc
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
import swifter

/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
user_app_usage_lines = 651007719
pieces = 10
interval = int(user_app_usage_lines / pieces)
uau_skiprows = [range(0, i*interval) for i in range(pieces)]
user_app_usage_path = '/Volumes/nvsd/user_app_usage.csv'
user_app_usage = None

In [3]:
with open(user_app_usage_path, 'r') as origin:
    for i in tqdm(range(pieces)):
        n_interval = interval if i != 9 else user_app_usage_lines - interval * 9
        with open('/Volumes/miguch/data/user_app_usage%d.csv'%i, 'w') as splitted:
            for k in tqdm(range(n_interval), leave=False):
                line = origin.readline()
                splitted.write(line)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/nvsd/user_app_usage.csv'

In [4]:
def process_date(d):
    ymd = d.split('-')
    if ymd[1] == '02':
        return 0
    else: return int(ymd[2])
def reprocess_user_app_usage(pieces_num):
    gc.collect()
    user_app_usage_part = pd.read_csv('/Volumes/miguch/data/user_app_usage%d.csv'%pieces_num, header=None)
    user_app_usage_part.columns = ['uid', 'appId', 'duration', 'times', 'use_date']
    user_app_usage_part['duration'] = user_app_usage_part['duration'].astype(np.int32)
    user_app_usage_part['times'] = user_app_usage_part['times'].astype(np.int16)
    user_app_usage_part['use_date'] = user_app_usage_part['use_date'].swifter.allow_dask_on_strings().apply(process_date)
    user_app_usage_part.to_csv('/Volumes/miguch/data/user_app_usages%d.csv'%pieces_num, header=None, index=False)
    
    

In [ ]:
from concurrent import futures
with futures.ProcessPoolExecutor() as pool:
  pool.map(reprocess_user_app_usage, range(pieces))
# for i in tqdm(range(pieces)):
#     reprocess_user_app_usage(i)